# Running Multiple Simulations With Partitions

## Overview

### Questions

* How can I partition a MPI communicator to run many independent simulations? 
* When are partitions useful?

### Objectives

* Show how to define a custom Communicator with more than one **partition**.
* Demonstrate how to use this to run many independent simulations with one `mpirun`.
* Explain how this is useful to **aggregate** jobs on HPC systems.

In [1]:
import os

fn = os.path.join(os.getcwd(), 'trajectory0.gsd')
![ -e "$fn" ] && rm "$fn"
fn = os.path.join(os.getcwd(), 'trajectory1.gsd')
![ -e "$fn" ] && rm "$fn"

## Partitioning communicators

So far in this tutorial you have seen how executing `mpirun -n 4 python3 script.py` will use domain decomposition to run *1* simulation split across 4 ranks via **domain decomposition**.
What if you wanted to run *2 different* simulations, each on *2* **ranks** with this command?
Or *4 different* simulations each on *1* **rank**?
This is called **partitioning** the MPI communicator.

In HOOMD-blue, you can do this by defining a non-default Communicator and specifying the ``ranks_per_partition`` argument.
Then you can use the ``communicator.partition`` as an identifier in your script to change input parameters.

In [2]:
%pycat hello_partition.py

import hoomd

communicator = hoomd.communicator.Communicator(ranks_per_partition = 2)
print(f'Hello from partition {communicator.partition} rank {communicator.rank}')


In [3]:
!mpirun -n 4 python3 hello_partition.py

Hello from partition 0 rank 1
Hello from partition 1 rank 0
Hello from partition 1 rank 1
Hello from partition 0 rank 0


In partitioned simulations, all ranks *within a given partition* must have the same input file, operations, and parameters.
However, the input file, operations and/or parameters can differ between partitions.
Use this to run many simulations with different temperatures, different initial configurations, or as will be shown in this example, different random number seeds.
Ensure that different partitions produce different output files.

In [4]:
%pycat lj_partition.py

import hoomd

communicator = hoomd.communicator.Communicator(ranks_per_partition = 2)

# Pass the communicator to the device.
device = hoomd.device.CPU(communicator=communicator)
sim = hoomd.Simulation(device=device)
sim.create_state_from_gsd(filename='random.gsd')

# Set the seed based on the partition
sim.seed = communicator.partition

integrator = hoomd.md.Integrator(dt=0.005)
cell = hoomd.md.nlist.Cell()
lj = hoomd.md.pair.LJ(nlist=cell)
lj.params[('A', 'A')] = dict(epsilon=1, sigma=1)
lj.r_cut[('A', 'A')] = 2.5
integrator.forces.append(lj)
langevin = hoomd.md.methods.Langevin(kT=1.5, filter=hoomd.filter.All())
integrator.methods.append(langevin)
sim.operations.integrator = integrator

# Use the partition id in the output file name.
gsd_writer = hoomd.write.GSD(filename=f'trajectory{communicator.partition}.gsd',
                             trigger=hoomd.trigger.Periodic(1000),
                             mode='xb')
sim.operations.writers.append(gsd_writer)

sim.run(1000)


In [5]:
!mpirun -n 4 python3 lj_partition.py

notice(2): Using domain decomposition: n_x = 1 n_y = 1 n_z = 2.
notice(2): Using domain decomposition: n_x = 1 n_y = 1 n_z = 2.


In [6]:
!ls trajectory?.gsd

trajectory0.gsd trajectory1.gsd


## Motivation

Why use partitions in simulations where you can just write one script with parameters and execute it more than once?
On some **HPC** systems, policies prefer fewer large jobs over many smaller jobs in the scheduler.
On these systems, you can obtain better throughput for your research when you use **partitions** to **aggregate** many independent simulations within one job.
While the details are beyond the scope of this tutorial, you may be interested in the aggregation feature of [signac-flow](https://docs.signac.io/projects/flow) which can automate the process.

In this section, you have learned how to run many simulations with different parameters using a single `mpirun` invocation.
This is the end of the MPI tutorial.